Da zaženete naslednje zvezke, če tega še niste storili, morate nastaviti ključ openai v datoteki .env kot `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Nato bomo naložili indeks vdelave v Pandas DataFrame. Indeks vdelave je shranjen v JSON datoteki z imenom `embedding_index_3m.json`. Indeks vdelave vsebuje vdelave za vsak YouTube prepis do konca oktobra 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Nato bomo ustvarili funkcijo z imenom `get_videos`, ki bo iskala v Indeksu vdelav za poizvedbo. Funkcija bo vrnila top 5 videoposnetkov, ki so najbolj podobni poizvedbi. Funkcija deluje tako:

1. Najprej se ustvari kopija Indeksa vdelav.
2. Nato se izračuna vdelava za poizvedbo z uporabo OpenAI Embedding API.
3. Nato se v Indeksu vdelav ustvari nov stolpec z imenom `similarity`. Stolpec `similarity` vsebuje kosinusno podobnost med vdelavo poizvedbe in vdelavo za vsak video segment.
4. Nato se Indeks vdelav filtrira po stolpcu `similarity`. Indeks vdelav se filtrira tako, da vključuje samo videoposnetke, ki imajo kosinusno podobnost večjo ali enako 0,75.
5. Na koncu se Indeks vdelav uredi po stolpcu `similarity` in vrne top 5 videoposnetkov.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Ta funkcija je zelo preprosta, samo izpiše rezultate iskalnega poizvedbe.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Najprej se indeks vdelave naloži v Pandas Dataframe.
2. Nato je uporabnik pozvan, da vnese poizvedbo.
3. Nato se pokliče funkcija `get_videos`, da se v indeksu vdelave poišče poizvedba.
4. Na koncu se pokliče funkcija `display_results`, da se uporabniku prikažejo rezultati.
5. Uporabnik je nato pozvan, da vnese novo poizvedbo. Ta postopek se nadaljuje, dokler uporabnik ne vnese `exit`.

![](../../../../translated_images/sl/notebook-search.1e320b9c7fcbb0bc.webp)

Pozvani boste, da vnesete poizvedbo. Vnesite poizvedbo in pritisnite enter. Aplikacija bo vrnila seznam videoposnetkov, ki so povezani s poizvedbo. Aplikacija bo prav tako vrnila povezavo do mesta v videoposnetku, kjer se nahaja odgovor na vprašanje.

Tukaj je nekaj poizvedb za preizkus:

- Kaj je Azure Machine Learning?
- Kako delujejo konvolucijske nevronske mreže?
- Kaj je nevronska mreža?
- Ali lahko uporabljam Jupyter Notebooks z Azure Machine Learning?
- Kaj je ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Omejitev odgovornosti**:
Ta dokument je bil preveden z uporabo storitve za prevajanje z umetno inteligenco [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas opozarjamo, da avtomatizirani prevodi lahko vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku velja za avtoritativni vir. Za ključne informacije priporočamo strokovni človeški prevod. Za morebitna nesporazume ali napačne interpretacije, ki izhajajo iz uporabe tega prevoda, ne odgovarjamo.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
